# Análise sobre os Preços dos Combustíveis no Brasil

Aluno: Rafael Pereira Cândido

Matrícula: 2221134

Curso: Ciência da Computação

Esse trabalho tem o propósito de analizar os preços dos combustíveis no Brasil do período de 2003 até 2023.

Chat sobre as queries e modelagem...: https://chatgpt.com/share/973eee34-e214-485c-8001-240b457834bf

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f
from uuid import uuid4
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
url = "jdbc:postgresql://localhost:5432/fuel_analysis"
properties = {
    "user": "root",
    "password": "root",
    "driver": "org.postgresql.Driver"
}

In [3]:
# PYSPARK UTILS
def create_uuid():
    return str(uuid4())

uuid_udf = f.udf(lambda: create_uuid(), StringType())



In [6]:
spark = SparkSession.builder \
    .appName("spark") \
    .master("local[*]") \
    .config("spark.jars", "/usr/local/spark/jars/postgresql-42.7.3.jar") \
    .getOrCreate()


In [9]:
df_postgres = spark.read.jdbc(url=url, table="test_table", properties=properties)
df_postgres.show()

Py4JJavaError: An error occurred while calling o35.jdbc.
: org.postgresql.util.PSQLException: The connection attempt failed.
	at org.postgresql.core.v3.ConnectionFactoryImpl.openConnectionImpl(ConnectionFactoryImpl.java:358)
	at org.postgresql.core.ConnectionFactory.openConnection(ConnectionFactory.java:54)
	at org.postgresql.jdbc.PgConnection.<init>(PgConnection.java:273)
	at org.postgresql.Driver.makeConnection(Driver.java:446)
	at org.postgresql.Driver.connect(Driver.java:298)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1(JdbcDialects.scala:160)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1$adapted(JdbcDialects.scala:156)
	at org.apache.spark.sql.execution.datasources.

In [ ]:
def extract_data(path):
  df = spark.read.csv(path, sep=';', inferSchema=True, header=True)
  
  return df

def transform_data(df):
  df = df.withColumn(
      "Data da Coleta",
      f.to_date(f.col("Data da Coleta").cast(StringType()), 'dd/MM/yyyy')
      )\
      .withColumn('Valor de Venda', f.regexp_replace('Valor de Venda', ',', '.'))\
      .withColumn('Valor de Venda', f.col('Valor de Venda').cast(DoubleType()))\
      .withColumn('Valor de Compra', f.regexp_replace('Valor de Compra', ',', '.'))\
      .withColumn('Valor de Compra', f.col('Valor de Compra').cast(DoubleType()))\
      .dropna(subset=['Valor de Venda', 'Valor de Compra'])\
      .fillna({'Complemento': 'S/C'})
      
  df = df.withColumnRenamed('Regiao - Sigla', 'Regiao_Sigla') \
       .withColumnRenamed('Estado - Sigla', 'Estado_Sigla') \
       .withColumnRenamed('CNPJ da Revenda', 'CNPJ_Revenda') \
       .withColumnRenamed('Data da Coleta', 'Data_Coleta') \
       .withColumnRenamed('Nome da Rua', 'Nome_Rua') \
       .withColumnRenamed('Numero Rua', 'Numero_Rua')\
       .withColumnRenamed('Valor de Venda', 'Valor_Venda') \
       .withColumnRenamed('Valor de Compra', 'Valor_Compra')
      
  return df

In [ ]:
base_path = '/home/jovyan/data/'

In [ ]:
path = base_path+'ca-2020-02.csv'
df = extract_data(path)
df = transform_data(df)

In [ ]:
df.show(5, False)

In [ ]:
dim_produto = df.select('Produto').distinct()\
                .withColumn('Produto_ID', uuid_udf())

# Criar DIM_Revenda
dim_revenda = df.select(f.col('CNPJ_Revenda'), f.col('Revenda').alias('Nome_Revenda')).distinct()\
                .withColumn('Revenda_ID', uuid_udf())

# Criar DIM_Tempo
dim_tempo = df.select('Data_Coleta').distinct()\
              .withColumn('Tempo_ID', uuid_udf())\
              .withColumn('Dia', f.dayofmonth(f.col('Data_Coleta')))\
              .withColumn('Mes', f.month(f.col('Data_Coleta')))\
              .withColumn('Ano', f.year(f.col('Data_Coleta')))\
              .withColumn('Dia_Semana', f.dayofweek(f.col('Data_Coleta')))

# Criar DIM_Localizacao e DIM_Localizacao_Geografica
dim_localizacao = df.select('Regiao_Sigla', 'Estado_Sigla', 'Municipio', 'Bairro', 'Cep').distinct()\
                    .withColumn('Localizacao_ID', uuid_udf())

# Criar DIM_Endereco
dim_endereco = df.select('Nome_Rua', 'Numero_Rua', 'Complemento').distinct()\
                 .withColumn('Endereco_ID', uuid_udf())


In [ ]:
dim_produto.show()
dim_revenda.show(5, False)
dim_tempo.show(5, False)
dim_localizacao.show(5, False)
dim_endereco.show(5, False)

In [ ]:
# Criar visualizações temporárias para os DataFrames
df.createOrReplaceTempView("df")
dim_produto.createOrReplaceTempView("dim_produto")
dim_revenda.createOrReplaceTempView("dim_revenda")
dim_tempo.createOrReplaceTempView("dim_tempo")
dim_localizacao.createOrReplaceTempView("dim_localizacao")
dim_endereco.createOrReplaceTempView("dim_endereco")
